<img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=250 style="padding: 10px"> 
<b>Little Demo: Catalog Queries with TAP </b> <br>
Contact authors: Gloria Fonseca Alvarez <br>
Last verified to run: 2024-08-12 <br>
LSST Science Pipelines version: Weekly  <br>
Container Size: medium <br>

This little demo shows how to query the DP0.2 catalogs using the TAP service. It is based on DP0.2 notebooks 02a and 02b. 

## 1. Set up

Import python packages and the Rubin Science Platform TAP service.

In [ ]:
import pandas
import numpy as np
import matplotlib.pyplot as plt

from lsst.rsp import get_tap_service, retrieve_query

pandas.set_option('display.max_rows', 20)

Instantiate the TAP service.

In [ ]:
service = get_tap_service("tap")

## 2. Explore the TAP schema and table columns.

Create a query to list all of the catalogs in the TAP schema.

In [ ]:
query = 'SELECT * FROM tap_schema.schemas'

Run the query.

In [ ]:
results = service.search(query)

In [ ]:
results

Store the results in an astropy table. 

In [ ]:
results = service.search(query).to_table()
results

List the tables in the DP0.2 catalogs.

In [ ]:
query = "SELECT * FROM tap_schema.tables " \
        "WHERE tap_schema.tables.schema_name = 'dp02_dc2_catalogs'" \
        "ORDER BY table_index ASC"
results = service.search(query).to_table()

In [ ]:
results

List all the columns in the Object table.

In [ ]:
query = "SELECT column_name, datatype, description, unit " \
        "FROM tap_schema.columns " \
        "WHERE table_name = 'dp02_dc2_catalogs.Object'"
results = service.search(query).to_table()

In [ ]:
results

List all the columns in the ForcedSource table.

In [ ]:
query = "SELECT column_name, datatype, description, unit " \
        "FROM tap_schema.columns " \
        "WHERE table_name = 'dp02_dc2_catalogs.ForcedSource'"
results = service.search(query).to_table()

In [ ]:
results

## 3. Query the Object table.

Define the coordinates and radius for a spatial search. 

In [ ]:
center_ra = 62
center_dec = -37
radius = 0.01

str_center_coords = str(center_ra) + ", " + str(center_dec)
str_radius = str(radius)

In [ ]:
query = "SELECT objectId, coord_ra, coord_dec, g_cModelFlux, g_cModelFluxErr "\
        "FROM dp02_dc2_catalogs.Object "\
        "WHERE CONTAINS(POINT('ICRS', coord_ra, coord_dec), "\
        "CIRCLE('ICRS', " + str_center_coords + ", " + str_radius + ")) = 1 "\
        "AND detect_isPrimary = 1"

In [ ]:
results = service.search(query).to_table()
results

Increase the search radius and run an asynchronous query.

In [ ]:
del results

In [ ]:
center_ra = 62
center_dec = -37
radius = 0.5

str_center_coords = str(center_ra) + ", " + str(center_dec)
str_radius = str(radius)

In [ ]:
query = "SELECT objectId, coord_ra, coord_dec, g_cModelFlux, g_cModelFluxErr "\
        "FROM dp02_dc2_catalogs.Object "\
        "WHERE CONTAINS(POINT('ICRS', coord_ra, coord_dec), "\
        "CIRCLE('ICRS', " + str_center_coords + ", " + str_radius + ")) = 1 "\
        "AND detect_isPrimary = 1"

In [ ]:
job = service.submit_job(query)
job.run()
job.wait(phases=['COMPLETED', 'ERROR'])
print('Job phase is', job.phase)
#job.raise_if_error()

In [ ]:
results = job.fetch_result().to_table()
results

Convert fluxes to magnitudes using the function scisql_nanojanskyToAbMab and scisql_nanojanskyToAbMagSigma.

In [ ]:
query = "SELECT coord_ra, coord_dec, "\
        "-2.5 * LOG10(g_cModelFlux) + 31.4 AS gmag1, "\
        "scisql_nanojanskyToAbMag(g_cModelFlux) AS gmag2, "\
        "scisql_nanojanskyToAbMagSigma(g_cModelFlux, g_cModelFluxErr) AS gmag2err "\
        "FROM dp02_dc2_catalogs.Object "\
        "WHERE CONTAINS(POINT('ICRS', coord_ra, coord_dec), "\
        "CIRCLE('ICRS', " + str_center_coords + ", " + str_radius + ")) = 1 "\
        "AND detect_isPrimary = 1"

In [ ]:
job = service.submit_job(query)
job.run()
job.wait(phases=['COMPLETED', 'ERROR'])
print('Job phase is', job.phase)

In [ ]:
results = job.fetch_result().to_table()
results

Get the g, r, and i magnitudes for point-like sources.

In [ ]:
query = "SELECT objectId, "\
        "scisql_nanojanskyToAbMag(g_psfFlux) AS gmag, "\
        "scisql_nanojanskyToAbMag(r_psfFlux) AS rmag, "\
        "scisql_nanojanskyToAbMag(i_psfFlux) AS imag "\
        "FROM dp02_dc2_catalogs.Object "\
        "WHERE CONTAINS(POINT('ICRS', coord_ra, coord_dec), "\
        "CIRCLE('ICRS', " + str_center_coords + ", " + str_radius + ")) = 1 "\
        "AND detect_isPrimary = 0 "\
        "AND g_extendedness = 0 "\
        "AND r_extendedness = 0 "\
        "AND i_extendedness = 0"

In [ ]:
job = service.submit_job(query)
job.run()
job.wait(phases=['COMPLETED', 'ERROR'])
print('Job phase is', job.phase)

In [ ]:
results = job.fetch_result().to_table()
#results

Plot a color-color diagram.

In [ ]:
fig = plt.figure(figsize=(4, 3))
plt.plot(results['gmag']-results['rmag'], results['rmag']-results['imag'],
         'o', mew=0, ms=3, alpha=0.1, color='black')
plt.xlim([-1, 2])
plt.ylim([-1, 2])
plt.xlabel('g-r color')
plt.ylabel('r-i color')
plt.show()

In [ ]:
query = "SELECT objectId, "\
        "scisql_nanojanskyToAbMag(g_psfFlux) AS gmag, "\
        "scisql_nanojanskyToAbMag(r_psfFlux) AS rmag, "\
        "scisql_nanojanskyToAbMag(i_psfFlux) AS imag "\
        "FROM dp02_dc2_catalogs.Object "\
        "WHERE CONTAINS(POINT('ICRS', coord_ra, coord_dec), "\
        "CIRCLE('ICRS', " + str_center_coords + ", " + str_radius + ")) = 1 "\
        "AND detect_isPrimary = 0 "\
        "AND g_extendedness = 0 "\
        "AND r_extendedness = 0 "\
        "AND i_extendedness = 0 "\
        "AND i_psfFlux/i_psfFluxErr > 5 "\
        "AND z_psfFlux/z_psfFluxErr > 5 "\
        "AND y_psfFlux/y_psfFluxErr > 5" 

In [ ]:
job = service.submit_job(query)
job.run()
job.wait(phases=['COMPLETED', 'ERROR'])
print('Job phase is', job.phase)

In [ ]:
results = job.fetch_result().to_table()
#results

In [ ]:
fig = plt.figure(figsize=(4, 3))
plt.plot(results['gmag']-results['rmag'], results['rmag']-results['imag'],
         'o', mew=0, ms=3, alpha=0.1, color='black')
plt.xlim([-1, 2])
plt.ylim([-1, 2])
plt.xlabel('g-r color')
plt.ylabel('r-i color')
plt.show()

## 4. Query the ForcedSource and CCD Visit tables.

Start with a known objectID. This is a variable star.

In [ ]:
str_target_objectId = '1651589610221899038'

Query using a table join. 

In [ ]:
query = "SELECT fs.objectId, fs.forcedSourceID, "\
        "cv.ccdVisitId, cv.band "\
        "FROM dp02_dc2_catalogs.ForcedSource AS fs "\
        "JOIN dp02_dc2_catalogs.CcdVisit AS cv ON fs.ccdVisitId = cv.ccdVisitId "\
        "WHERE fs.objectId = " + str_target_objectId

In [ ]:
job = service.submit_job(query)
job.run()
job.wait(phases=['COMPLETED', 'ERROR'])
print('Job phase is', job.phase)

In [ ]:
results = job.fetch_result().to_table()
results

Plot the i-band lightcurve.

In [ ]:
query = "SELECT fs.objectId, fs.forcedSourceID, "\
        "scisql_nanojanskyToAbMag(fs.psfFlux) AS mag, "\
        "scisql_nanojanskyToAbMagSigma(fs.psfFlux, fs.psfFluxErr) AS magerr, "\
        "cv.band, cv.expMidptMJD "\
        "FROM dp02_dc2_catalogs.ForcedSource AS fs "\
        "JOIN dp02_dc2_catalogs.CcdVisit AS cv ON fs.ccdVisitId = cv.ccdVisitId "\
        "WHERE fs.objectId = " + str_target_objectId + "  "\
        "AND cv.band = 'i'"

In [ ]:
job = service.submit_job(query)
job.run()
job.wait(phases=['COMPLETED', 'ERROR'])
print('Job phase is', job.phase)

In [ ]:
results = job.fetch_result().to_table()
results

In [ ]:
fig = plt.figure(figsize=(4, 3))
plt.plot(results['expMidptMJD'], results['mag'],'o', color='#850000', alpha=0.5, mew=0, label='i')
plt.gca().invert_yaxis()
plt.xlabel('Modified Julian Date')
plt.ylabel('Apparent Magnitude')
plt.legend(loc='upper right', handletextpad=0)
plt.show()